### Results for: "Efficient and reproducible pipelines for spike sorting large-scale electrophysiology data"

This notebook reproduces the panels for Figures 4, 5 and S1 of the *Efficient and reproducible pipelines for spike sorting large-scale electrophysiology data* manuscript.

In [ ]:
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from itertools import combinations
from functools import reduce

import warnings

from utils import plot_aggregated_results, plot_performance_curves, stat_test

%matplotlib ipympl

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
save_figs = True

In [ ]:
data_folder = Path("../data")
results_folder = Path("../results")

# Sorters

In [ ]:
figures_sorters = results_folder / "sorters"
figures_sorters.mkdir(parents=True, exist_ok=True)

In [ ]:
np1_metrics = pd.read_csv(data_folder / "sorters" / "np1" / "metrics_sorted.csv", index_col=False)
np2_metrics = pd.read_csv(data_folder / "sorters" / "np2" / "metrics_sorted.csv", index_col=False)
all_metrics_sorters = pd.concat([np1_metrics, np2_metrics])

In [ ]:
colors_sorters = {
    "kilosort25": "C3",
    "kilosort4": "C4"
}
order_sorters = np.unique(all_metrics_sorters["sorting_case"])

In [ ]:
metric_res_name = "rp_contamination"
fig, ax = plt.subplots(figsize=(12, 7))
sns.boxenplot(data=all_metrics_sorters, y=metric_res_name, x="sorting_case",  palette=colors_sorters, ax=ax, showfliers=False, log_scale=True,
              order=order_sorters)
sns.despine(fig)
ax.legend().remove()
ax.set_ylabel("")
ax.set_xlabel("")
ax.set_xticklabels([])
if save_figs:
    fig.savefig(figures_sorters / "rp_violations.pdf")

In [ ]:
metric_res_name = "presence_ratio"
fig, ax = plt.subplots(figsize=(12, 7))
sns.histplot(data=all_metrics_sorters, x=metric_res_name, hue="sorting_case", stat="probability", palette=colors_sorters, bins=20, fill=False, ax=ax)
sns.despine(fig)
ax.legend().remove()
ax.set_ylabel("")
ax.set_xlabel("")
if save_figs:
    fig.savefig(figures_sorters / "presence_ratio.pdf")

In [ ]:
test_res = stat_test(all_metrics_sorters, column_group_by="sorting_case", test_columns=["rp_violations", "presence_ratio"], paired=False, verbose=True)

In [ ]:
all_metrics_sorters.groupby("sorting_case")["sorting_case"].count()

In [ ]:
dataframes_sorters_all = {}

for probe in ["np1", "np2"]:
    dataframes_sorters_all[probe] = {}
    df_folder = data_folder / "sorters" / probe
    dataframes = {}
    for csv_file in df_folder.iterdir():
        if csv_file.suffix != ".csv":
            continue
        df_name = csv_file.stem
        df = pd.read_csv(csv_file, index_col=False)
        dataframes[df_name] = df
    dataframes_sorters_all[probe] = dataframes

In [ ]:
figs_probe = {}
for probe, dataframes in dataframes_sorters_all.items():
    figs, dfs_merged = plot_aggregated_results(dataframes, colors_sorters, plot_metrics=True)
    # test
    print(f"\n\nTests for probe {probe}:")
    test_res = stat_test(dataframes["performances"], column_group_by="sorting_case", test_columns=["accuracy", "precision", "recall"], paired=True, verbose=True)
    figs_probe[probe] = figs
    if save_figs:
        for fname, fig in figs.items():
            fig.savefig(figures_sorters / f"{probe}_{fname}.pdf")
            fig.savefig(figures_sorters / f"{probe}_{fname}.png", dpi=300)

In [ ]:
# plot run times
for probe, df_all in dataframes_sorters_all.items():
    print(f"Probe {probe}")
    df_runtimes = df_all["run_times"]
    print(df_runtimes.groupby("sorting_case").count()["run_times"])
    df_runtimes.loc[:, "run_times_rel"] = df_runtimes["run_times"] /  df_runtimes["duration"]
    fig, ax = plt.subplots(figsize=(7, 7))
    sns.boxenplot(data=df_runtimes, y="run_times_rel", x="sorting_case",  palette=colors_sorters, ax=ax, showfliers=True, log_scale=False,
                  order=order_sorters)
    sns.despine(fig)
    ax.legend().remove()
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.set_xticklabels([])
    if save_figs:
        fig.savefig(figures_sorters / f"run_times_{probe}.pdf")

# Lossy

In [ ]:
figures_lossy = results_folder / "lossy"
figures_lossy.mkdir(parents=True, exist_ok=True)

In [ ]:
np1_metrics = pd.read_csv(data_folder / "lossy" / "np1" / "metrics_sorted.csv", index_col=False)
np2_metrics = pd.read_csv(data_folder / "lossy" / "np2" / "metrics_sorted.csv", index_col=False)

In [ ]:
all_metrics_lossy = pd.concat([np1_metrics, np2_metrics])

In [ ]:
all_metrics_lossy.groupby("sorting_case")["sorting_case"].count()

In [ ]:
order_lossy = np.unique(all_metrics_lossy["sorting_case"])
order_lossy = [order_lossy[0]] + list(order_lossy[1:][::-1])
colors_lossy = {}
cmap_is = np.linspace(0.2, 0.6, len(order_lossy))
cmap = "Greens_r"
for i, s in enumerate(order_lossy[1:]):
    cmap = plt.get_cmap(cmap)
    colors_lossy[s] = cmap(cmap_is[i])
colors_lossy["lossless"] = "C1"

In [ ]:
metric_res_name = "rp_contamination"
fig, ax = plt.subplots(figsize=(12, 7))
sns.boxenplot(data=all_metrics_lossy, y=metric_res_name, x="sorting_case",  palette=colors_lossy, ax=ax, showfliers=False, log_scale=True,
              order=order_lossy)
sns.despine(fig)
ax.legend().remove()
ax.set_ylabel("")
ax.set_xlabel("")
ax.set_xticklabels([])
if save_figs:
    fig.savefig(figures_lossy / "rp_violations.pdf")

In [ ]:
metric_res_name = "presence_ratio"
fig, ax = plt.subplots(figsize=(12, 7))
sns.histplot(data=all_metrics_lossy, x=metric_res_name, hue="sorting_case", stat="probability", palette=colors_lossy, bins=20, fill=False, ax=ax)
sns.despine(fig)
ax.legend().remove()
ax.set_ylabel("")
ax.set_xlabel("")
# ax.set_xlim(-0.02, 0.3)
if save_figs:
    fig.savefig(figures_lossy / "presence_ratio.pdf")

In [ ]:
test_res = stat_test(all_metrics_lossy, column_group_by="sorting_case", test_columns=["rp_violations", "presence_ratio"], verbose=True, sig=0.01)

In [ ]:
dataframes_lossy_all = {}

for probe in ["np1", "np2"]:
    dataframes_lossy_all[probe] = {}
    df_folder = data_folder / "lossy" / probe
    dataframes = {}
    for csv_file in df_folder.iterdir():
        if csv_file.suffix != ".csv":
            continue
        df_name = csv_file.stem
        df = pd.read_csv(csv_file, index_col=False)
        dataframes[df_name] = df
    dataframes_lossy_all[probe] = dataframes

In [ ]:
figs_probe = {}
for probe, dataframes in dataframes_lossy_all.items():
    figs, _ = plot_aggregated_results(dataframes, colors_lossy, include_string_in_pair="lossless", plot_metrics=False)
    # add Kilosort25 line
    axes_perf = figs["performance"].get_axes()
    df_units_ks25 = dataframes_sorters_all[probe]["performances"].query("sorting_case == 'kilosort25'")
    _ = plot_performance_curves(df_units_ks25, ["accuracy", "precision", "recall"], ["kilosort25"], colors_sorters, axes_perf, lw=1.5, ls="--", alpha=0.8)

    figs_probe[probe] = figs
    print(f"\n\nTests for probe {probe}:")
    # we have to make sure counts are the same
    df = dataframes["performances"]
    case_counts = df["sorting_case"].value_counts()
    if len(set(case_counts.values)) > 1:
        print("Filtering entries to ensure paired samples")
        min_case = case_counts.idxmin()
        # Step 3: Extract the combinations from the smallest case
        valid_keys = df[df["sorting_case"] == min_case][["stream_name", "session", "case", "gt_unit_id"]].drop_duplicates()
        # Step 4: Filter the full DataFrame to keep only rows with those combinations
        filtered_df = df.merge(valid_keys, on=["stream_name", "session", "case", "gt_unit_id"], how="inner")
    else:
        filtered_df = df
    perfs_df = dataframes["performances"]
    test_res = stat_test(filtered_df, column_group_by="sorting_case", test_columns=["accuracy", "precision", "recall"], paired=True, verbose=True)
    if save_figs:
        for fname, fig in figs.items():
            fig.savefig(figures_lossy / f"{probe}_{fname}.pdf")

## Plot accuracy VS compression ratio

In [ ]:
compression_ratios = {
    "np1": {
        "lossless": 3.53,  # 1 / 0.2839 (avg. file size)
        "wavpack-3": 5.33,
        "wavpack-2.5": 6.4,
        "wavpack-2.25": 7.1
    },
    "np2": {
        "lossless": 3.51, # 1 / 0.2851
        "wavpack-3": 5.33,
        "wavpack-2.5": 6.4,
        "wavpack-2.25": 7.1
    },
}

In [ ]:
dataframes_lossy_all.keys()

In [ ]:
from matplotlib.lines import Line2D


figs_probe = {}
marker_size = 50
lw = 2
for probe, dataframes in dataframes_lossy_all.items():
    fig, ax = plt.subplots()
    med_accuracy = dataframes["performances"].groupby("sorting_case")["accuracy"].median()
    med_precision = dataframes["performances"].groupby("sorting_case")["precision"].median()
    med_recall = dataframes["performances"].groupby("sorting_case")["recall"].median()

    keys = list(compression_ratios[probe].keys())
    crs = list(compression_ratios[probe].values())
    # accuracy
    ax.plot(crs, med_accuracy[compression_ratios[probe].keys()], color="grey",  ls="solid", lw=lw)
    ax.scatter(crs, med_accuracy[compression_ratios[probe].keys()], 
               c=[colors_lossy[k] for k in keys], s=marker_size, zorder=10, marker="d", edgecolors="k")
    # precision
    ax.plot(crs, med_precision[compression_ratios[probe].keys()], color="grey", ls="dashed", lw=lw)
    ax.scatter(crs, med_precision[compression_ratios[probe].keys()], 
               c=[colors_lossy[k] for k in keys], s=marker_size, zorder=10, marker="o", edgecolors="k")
    # recall
    ax.plot(crs, med_recall[compression_ratios[probe].keys()], color="grey", ls="dotted", lw=lw)
    ax.scatter(crs, med_recall[compression_ratios[probe].keys()], 
               c=[colors_lossy[k] for k in keys], s=marker_size, zorder=10, marker="^", edgecolors="k")

    ax.set_ylim(0, 1)
    ax.set_ylabel("Performance")
    ax.set_xlabel("Compression ratio")
    ax.set_xticks(crs)
    ax.set_xticklabels([f"{label}\n({cr.replace('wavpack', 'BPS')})" for cr, label in compression_ratios[probe].items()])
    
    # add legend
    # Other line handles
    acc_handle = Line2D([0], [0], color='grey', marker='d', 
                        markeredgecolor="k", linestyle='solid', label='Accuracy', lw=lw)
    prec_handle = Line2D([0], [0], color='grey', marker='o', 
                         markeredgecolor="k", linestyle='dashed', label='Precision', lw=lw)
    rec_handle = Line2D([0], [0], color='grey', marker='^', 
                        markeredgecolor="k", linestyle='dotted', label='Recall', lw=lw)

    
    # Legend
    plt.legend(handles=[acc_handle, prec_handle, rec_handle])
    
    fig.subplots_adjust(bottom=0.2)
    sns.despine(fig)
    if save_figs:
        fig.savefig(figures_lossy / f"{probe}_perf_cr.pdf")